<a href="https://colab.research.google.com/github/jamespan1118/Xenopus_signal_processing/blob/main/HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Creating the folder structures from raw data

In [26]:
import pandas as pd
import numpy as np
import os

In [22]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [23]:
worksheet = gc.open('Male Female Assay Log').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df = pd.DataFrame.from_records(rows)
df.columns = df.iloc[0] 
df = df[1:]
df.head()

,Date,Separation Status,Recording_Tank,Hydraphone,Folder Name,Tank_Number_male,RFID_male,Species_male,Mass_male,hCG_injected_male,Tank_Number_Female,RFID_Female,Species_Female,Mass_Female,hCG_injected_Female,Comment,Frog Camera
1,06/14/22,Separate,1,1,20220614_F827E-F829C_F8340-F8297_F835F-F82C0,,F829C,laevis,50.1,0.2505,31,F837E,laevis,130.3,0.6515,"only recorded for 55 minutes, terminated unexp...",
2,06/14/22,Separate,2,2,20220614_F827E-F829C_F8340-F8297_F835F-F82C0,,F8297,laevis,46,0.23,31,F8340,laevis,98.7,0.4935,"only recorded for 55 minutes, terminated unexp...",
3,06/14/22,Separate,3,3,20220614_F827E-F829C_F8340-F8297_F835F-F82C0,,F82C0,laevis,56.3,0.2815,31,F835F,laevis,102.7,0.5135,"only recorded for 55 minutes, terminated unexp...",
4,06/15/22,Together,1,1,20220615_F827E-F829C_F8340-F8297_F835F-F82C0,,F829C,laevis,50.1,0.2505,31,F837E,laevis,130.3,0.6515,ticking found,
5,06/15/22,Together,2,2,20220615_F827E-F829C_F8340-F8297_F835F-F82C0,,F8297,laevis,46,0.23,31,F8340,laevis,98.7,0.4935,no ticking,


In [40]:
def parse_folder_name(folder_name):
  pair_number = folder_name.count('_')
  pair_names = []
  date = folder_name.split('_')[0]
  for i in range(0,pair_number):
    pair_name = folder_name.split('_')[i+1]
    pair_names.append(pair_name)
  #search for species
  species_list = []
  for i in pair_names:
    female_ID = i.split('-')[1]
    #species = np.array(df.loc[df['RFID_Female']==female_ID]['Species_Female'])[0]
    species = df.loc[df['RFID_Female']==female_ID]['Species_Female'].values[0]
    species_list.append(species)
  assert(len(pair_names)==len(species_list))
  return pair_number, pair_names, species_list

In [41]:
number,names,species_list = parse_folder_name('20220713_F832F-F8361_AD812-AD810_AD7ED-AD7F8_AD7F5-AD817')
print(number)
print(len(names))
print(len(species_list))

4
4
4


In [50]:
overall_dir = os.path.dirname(os.path.realpath('/content/drive/MyDrive/overall/Male Female Assay Log.gsheet'))

In [65]:
def make_pair_dir(folder_name,overall_dir):
  pair_number, pair_names, species_list = parse_folder_name(folder_name)
  for i in range(0,pair_number):
    current_species = species_list[i]
    species_dir = os.path.join(overall_dir,current_species)
    os.chdir(species_dir)
    pair_name = pair_names[i]
    pair_dir = os.path.join(species_dir,pair_name)
    if not os.path.exists(pair_dir):
      os.mkdir(pair_dir)

In [67]:
make_pair_dir('20220624_ F820D-F820C_F8218-F8215_F8244-F8211',overall_dir)